# Subset Notebook

Working towards analyzing clusters derived in the cluster notebook so that they can be used to create RAG vectors

Generates gene signatures for subsequent analysis.


Input:

* data/subset.h5ad

Output:

* data/sigs.csv
* data/sigs.parquet

In [1]:
import warnings

# import numba
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning

warnings.filterwarnings("ignore", category=DeprecationWarning)

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", category=NumbaPendingDeprecationWarning)

In [2]:
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as ad

# import os
from scipy.sparse import csr_matrix
import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

sc.set_figure_params(figsize=(5, 5))  # type: ignore

In [3]:
#
# load the AnnData containing the data processed according to the Best Practices tutorial
#
adata = sc.read_h5ad("../data/subset.h5ad")
adata

AnnData object with n_obs × n_vars = 9370 × 31208
    obs: 'n_genes_by_counts', 'total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'DF_score', 'batch', 'size_factors', 'leiden_2'
    var: 'gene_ids', 'feature_types', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable'

In [4]:
def get_highly_variable_genes(adata: ad.AnnData) -> ad.AnnData:
    """
    get_highly_variable_genes 

    Get the subset of genes in adata that have been classified as highly variable

    Args:
        adata (ad.AnnData): The input.

    Returns:
        ad.AnnData: A subset containing only highly variable genes
    """
    b = adata.var[adata.var.highly_variable]
    return adata[:, b.index] # type: ignore

#
# create an AnnData with only highly variable genes
hvar = get_highly_variable_genes(adata)
hvar

View of AnnData object with n_obs × n_vars = 9370 × 4000
    obs: 'n_genes_by_counts', 'total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'DF_score', 'batch', 'size_factors', 'leiden_2'
    var: 'gene_ids', 'feature_types', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable'

In [5]:
def get_cluster_names(adata: ad.AnnData, criterion="leiden_2") -> list[str]:
    """
    get_cluster_names 

    Get the cluster names assigned by the given criterion.

    Args:
        adata (ad.AnnData): The input.
        criterion (str, optional): The criterion column used to cluster the data. Defaults to "leiden_2".

    Returns:
        list[str]: The list of cluster names.
    """
    clusters = [
        str(x) for x in sorted([int(cluster) for cluster in adata.obs[criterion].unique()])
    ]
    return clusters
print(get_cluster_names(hvar))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']


In [6]:
from typing import Any

def partition_clusters(adata: ad.AnnData, criterion="leiden_2") -> dict[str, ad.AnnData]:
    """
    partition_clusters 
    Divide up the data into a series of subset AnnDatas stored in a dictionary
    where the key is the cluster name.

    Calls get_cluster_names

    Args:
        adata (ad.AnnData): The input.
        criterion (str, optional): The column containing the cluster information. Defaults to "leiden_2".

    Returns:
        dict[str, ad.AnnData]: A dictionary with cluster names as keys and data in AnnData format. 
    """
    clusters = get_cluster_names(adata, criterion)
    cluster_table: dict[str, Any] = {}
    for cluster in clusters:
        subset = adata[adata.obs[criterion] == cluster] # type: ignore
        cluster_table[cluster] = subset.copy()
    return cluster_table

cluster_table = partition_clusters(hvar)
print(f"Length of cluster table: {len(cluster_table)}")
assert isinstance(cluster_table[list(cluster_table.keys())[0]], ad.AnnData) # ensure that we are dealing with copies, not slices
assert len(cluster_table) == len(get_cluster_names(adata))

Length of cluster table: 19


In [7]:
def calculate_highest_frequency_genes(adata: ad.AnnData, number_of_genes:int = 20, expression_threshold=0.0, verbose=False) -> list[str]:
    """
    calculate_highest_frequency_genes 

    Determine the genes that are most frequently in cells.

    Args:
        adata (ad.AnnData): The input.
        number_of_genes (int, optional): The threshold to be called high frequency. Defaults to 20.
        expression_threshold (float, optional): The minimum expression level for a gene to be considered. Defaults to 0.0.
        verbose (bool, optional): Prints additional information, if True. Defaults to False.

    Returns:
        list[str]: A list of gene names representing the genes expressed in the most cells. 
    """
    cell_count, gene_count = adata.shape
    if verbose:
        print(f"{cell_count} cells, {gene_count} genes")
    gene_table = {}

    
    # create a table with the genes as the keys and frequency as the value
    for cell_no in range(cell_count):
        b = adata.X[cell_no] > expression_threshold # type: ignore
        genes = adata.var.index[b]
        for gene in genes:
            if gene in gene_table:
                gene_table[gene] += 1
            else:
                gene_table[gene] = 1
    # set the table to be in descending order 
    gene_table = dict(sorted(gene_table.items(), key=lambda x:x[1], reverse=True))

    # create a list of the most frequently expressed in cells
    gene_list = list(gene_table.keys())[0:number_of_genes]
    if verbose:
        for gene in gene_list:
            print(f"{gene}:{gene_table[gene]} ({gene_table[gene]/cell_count*100:4.1f}%)")
    return gene_list
            

In [14]:
def calculate_gene_signature_per_cluster(cluster_table: dict[str, ad.AnnData], 
                                         genes_per_cluster=25,
                                         repeat_limit=5,
                                         expression_threshold=0.0
                                        )-> dict[str,list[str]]:
    """
    calculate_gene_signature_per_cluster 

    Calculate a cluster specific gene signature based on the most highly expressed genes per cell.

    Calls calculate_highest_frequency_genes

    Args:
        cluster_table (dict[str, ad.AnnData]): A table with cluster names as the key, cluster specific data as the input.
        genes_per_cluster (int, optional): Number of genes to select by cluster. Defaults to 25.
        repeat_limit (int, optional): The maximum number of clusters with a given gene. Defaults to 5.
        expression_threshold (float, optional): Minimum expression threshold for each gene. Defaults to 0.0.

    Returns:
        dict[str,list[str]]: _description_
    """
    gene_dict = {}

    for cluster in cluster_table:
        cdata = cluster_table[cluster]
        gene_list = calculate_highest_frequency_genes(
                                                    adata=cdata, 
                                                    number_of_genes=genes_per_cluster, 
                                                    expression_threshold=expression_threshold)
        # print(f"Cluster:{cluster}. Genes: {gene_list}")
        for gene in gene_list:
            if gene in gene_dict:
                gene_dict[gene].append(cluster)
            else:
                gene_dict[gene] = [cluster]
    # eliminate genes that are present "everywhere" (i.e. in multiple clusters)
    # in the following len(v) represents the number of clusters expressing the gene k
    gene_dict = {k:v for k,v in gene_dict.items() if len(v) < repeat_limit}

    # now calcuate the gene list for each cluster
    cluster_dict = {k:list() for k in cluster_table}
    for gene in gene_dict:
        clusters = gene_dict[gene]
        for cluster in clusters:
            cluster_dict[cluster].append(gene)
    return cluster_dict

cluster_dict = calculate_gene_signature_per_cluster(cluster_table, expression_threshold=1.7, repeat_limit=6)
for cluster in cluster_dict:
    print(f"{cluster}:{cluster_dict[cluster]}")
#
# save the result as a dataframe
#
df = pd.DataFrame()
df['cluster'] = [str(x) for x in cluster_dict.keys()]
df['signature'] = [" ".join(x) for x in cluster_dict.values()]
df['embeddings'] = ['' for x in range(len(cluster_dict))]
df['cell_id'] = df.index
print(df.shape)
df.to_parquet("../data/clustered.parquet")
print("file written")



0:['GNLY', 'TXNIP', 'S100A4', 'PRKCH', 'SYNE2']
1:['PRKCH', 'SYNE2', 'RIPOR2', 'SYNE1', 'THEMIS', 'ANKRD44', 'FYN', 'TGFBR3', 'PARP8', 'AOAH']
2:['SYNE2', 'PARP8', 'CBLB', 'RORA', 'AL136456.1', 'IL7R', 'PLCB1', 'SKAP1']
3:['GNLY', 'TXNIP', 'S100A4', 'SYNE1', 'DDX5', 'HLA-A']
4:['RIPOR2', 'ANKRD44', 'SKAP1', 'FOXP1', 'PDE3B', 'INPP4B', 'ZBTB20', 'MAML2', 'ATM', 'SMCHD1', 'NELL2']
5:['ANKRD44', 'FOXP1', 'SMCHD1', 'BACH2', 'BANK1', 'RALGPS2', 'FCHSD2', 'ADK', 'ZCCHC7', 'CAMK2D', 'LYN', 'MEF2C']
6:['RIPOR2', 'ANKRD44', 'FOXP1', 'ZBTB20', 'SMCHD1', 'BACH2', 'BANK1', 'RALGPS2', 'FCHSD2', 'ADK', 'ZCCHC7', 'CAMK2D', 'LYN', 'MEF2C', 'ARHGAP24', 'PRKCB']
7:['PRKCH', 'FYN', 'PARP8', 'AOAH', 'CBLB', 'SKAP1', 'ZBTB20', 'CEMIP2', 'CNOT6L', 'ATP8A1', 'TOX']
8:['FOXP1', 'BACH2', 'ZCCHC7', 'LYN', 'PCDH9', 'PDE4D', 'SSBP2', 'ACSM3', 'TCF4', 'ARID1B', 'EBF1', 'TMEM131L', 'SYK', 'MIR181A1HG', 'IGHM']
9:['AOAH', 'LYN', 'NAMPT', 'VCAN', 'ANXA1', 'NEAT1', 'DPYD', 'ARHGAP26', 'PLXDC2', 'QKI', 'FOS', 'ATP2B1',

In [32]:
def find_common_genes(adata: ad.AnnData, genes_per_cluster=25, repeat_limit=5, expression_threshold=0.0) -> set[str]:
    """
    find_common_genes 

    Find genes that are almost ubiquitously expressed.

    Args:
        adata (ad.AnnData): The input.
        genes_per_cluster (int, optional): The number of genes per cluster to consider. Defaults to 25.
        repeat_limit (int, optional): The maximum number of genes allowed in multiple clusters. Defaults to 5.
        expression_threshold (float, optional): The minimum gene expression threshold. Defaults to 0.0.

    Returns:
        set[str]: A set of genes that are widely expressed for potential elimination from gene signatures.
    """
    gene_dict = {}
    for cluster in cluster_table:
        cluster_adata = cluster_table[cluster]
        gene_list = calculate_highest_frequency_genes(
                                                        adata=cluster_adata, 
                                                        number_of_genes=genes_per_cluster,
                                                        expression_threshold=expression_threshold)
        # record which clusters express each gene
        for gene in gene_list:
            if gene in gene_dict:
                gene_dict[gene].append(cluster)
            else:
                gene_dict[gene] = [cluster]
    # filter out genes that are only present in a few clusters
    gene_dict = {k:v for k,v in gene_dict.items() if len(v) >= repeat_limit}

    # get the resulting list of gene names
    gene_names = list(gene_dict.keys())
    common_genes = set(gene_names)
    
    return common_genes
        

find_common_genes(adata, expression_threshold=1.5)

{'ACTB',
 'AFF3',
 'ANKRD44',
 'ARHGAP15',
 'B2M',
 'CD74',
 'EEF1A1',
 'HBB',
 'HLA-B',
 'MALAT1',
 'MBNL1',
 'MT-ATP6',
 'MT-CO1',
 'MT-CO2',
 'MT-CO3',
 'MT-CYB',
 'MT-ND4',
 'MT-ND5',
 'PLCG2',
 'PTPRC',
 'RABGAP1L',
 'UTRN',
 'ZBTB20',
 'ZEB2'}

In [10]:
#
# need to extract data for a single cell
#
hv = get_highly_variable_genes(adata)
cell_name = hv.obs.index[4]
print(cell_name)
gene_data = hv.X[4].copy() # type: ignore
print(len(gene_data))



TGACCAAGTAGACAAA
4000


In [35]:
expression_threshold = 1.5
b= gene_data > expression_threshold
expression = gene_data[b]
names = hv.var.index[b]
assert expression.shape == names.shape
redundant = find_common_genes(hv, expression_threshold=expression_threshold)
genes = dict(sorted(dict(zip(names,expression)).items(), key=lambda x:x[1], reverse=True))
genes = {k:v for k,v in genes.items() if k not in redundant and not k.startswith("MT-")}
# genes = {k:v for k,v in genes.items() if k not in redundant }
print(len(genes))


15


In [36]:
genes

{'RPL11': 1.8190973713412917,
 'CEP350': 1.8190973713412917,
 'GNLY': 1.8190973713412917,
 'PTPN4': 1.8190973713412917,
 'SMARCA5': 1.8190973713412917,
 'KIAA0825': 1.8190973713412917,
 'ORC5': 1.8190973713412917,
 'SARAF': 1.8190973713412917,
 'PDCD4': 1.8190973713412917,
 'ABLIM1': 1.8190973713412917,
 'FNBP4': 1.8190973713412917,
 'SLC38A1': 1.8190973713412917,
 'ZC3H13': 1.8190973713412917,
 'CTDSPL2': 1.8190973713412917,
 'NF1': 1.8190973713412917}

In [37]:
def get_gene_signature(adata: ad.AnnData,
                   feature_index: int,
                   expression_threshold=0.0,
                   redundant_genes: set[str]=set(),
                   verbose: bool = False) -> dict[str,float]:
    """Calculate the list of genes for the given cell based on the feature_index.  Only non-mitochondrial
    genes with an expression level greater than the provided expression threshold are reported.

    Args:
        adata (ad.AnnData): The AnnData containing the cells.  Assume that only highly variable genes have been provided.
        feature_index (int): The index of the cell to be measured
        expression_threshold (float, optional): The minimum expression level. Defaults to 0.0, which returns all non-zero genes.
        redundant_genes (set[str], optional): Genes to be filtered out from the final result. Defaults to the empty set.
        verbose (bool, optional): Print intermediated data to standard output. Defaults to False.
    Returns:
        dict[str, float]: Dictionary with gene names as keys and expression as values.
    """
    num_cells, _ = adata.shape
    if feature_index < 0 or feature_index > num_cells-1:
        raise ValueError(f"Feature index ({feature_index}) outside the range of cells (0..{num_cells-1}) in the current dataset.")
    gene_data = adata.X[feature_index].copy() # type: ignore
    if verbose:
        print(f"Started with {len(gene_data)} genes")
    # calculate the mask to find gene subset
    b= gene_data > expression_threshold
    expression = gene_data[b]
    names = adata.var.index[b] 
    assert expression.shape == names.shape
    if verbose:
        print(f"Found {len(names)} genes exceeding expression threshold.")
    
    # sort the genes based on expression
    genes = dict(sorted(dict(zip(names,expression)).items(), key=lambda x:x[1], reverse=True))
    # remove redundant and mitochondrial genes
    genes = {k:v for k,v in genes.items() if k not in redundant_genes and not k.startswith("MT-")}
    if verbose:
        print(f"Found {len(genes)} genes after filtering.")
    return genes

In [38]:
expression_threshold = 1.5
redundant = find_common_genes(hv, expression_threshold=expression_threshold)
sig = get_gene_signature(adata=hv, 
                     feature_index=2, 
                     expression_threshold=expression_threshold, 
                     redundant_genes=redundant,
                     verbose=False)
print(len(sig))
sig

44


{'BANK1': 2.4271888792164433,
 'ARID1B': 2.4271888792164433,
 'PLEKHA2': 2.3061907112814617,
 'LINC01619': 2.1685103296015886,
 'BBX': 2.0088007503141467,
 'LIX1-AS1': 2.0088007503141467,
 'BACH2': 2.0088007503141467,
 'JAZF1': 2.0088007503141467,
 'JMJD1C': 2.0088007503141467,
 'CHD9': 2.0088007503141467,
 'COBLL1': 1.8186444515907547,
 'ITPR1': 1.8186444515907547,
 'ZSWIM6': 1.8186444515907547,
 'FBXL17': 1.8186444515907547,
 'CDK14': 1.8186444515907547,
 'ADK': 1.8186444515907547,
 'FCHSD2': 1.8186444515907547,
 'PRH1': 1.8186444515907547,
 'ZFAND6': 1.8186444515907547,
 'PRKCB': 1.8186444515907547,
 'ANKRD12': 1.8186444515907547,
 'COP1': 1.5836324776071877,
 'RALGPS2': 1.5836324776071877,
 'USP34': 1.5836324776071877,
 'IWS1': 1.5836324776071877,
 'MGAT5': 1.5836324776071877,
 'SP100': 1.5836324776071877,
 'FOXP1': 1.5836324776071877,
 'ACAP2': 1.5836324776071877,
 'TAPT1': 1.5836324776071877,
 'TMEM131L': 1.5836324776071877,
 'EBF1': 1.5836324776071877,
 'GMDS-DT': 1.583632477607

In [15]:
expression_threshold = 1.5
redundant = find_common_genes(hv, expression_threshold=expression_threshold)
dataframes :list[pd.DataFrame] = []
total_cells=0
for cluster_name in cluster_table:
    # test with a single cluster
    cluster_signatures = pd.DataFrame(columns = ['cluster','signature'], index=cluster_table[cluster_name].obs.index)
    cluster_adata = cluster_table[cluster_name]
    n_cells, n_genes = cluster_adata.shape
    print(n_cells)
    total_cells+=n_cells
    # for cell_no in range(10):
    for cell_no in range(n_cells):
        cluster_signatures.iloc[cell_no,0] = cluster_name
        cluster_signatures.iloc[cell_no,1] = " ".join(list(get_gene_signature(adata=cluster_adata, 
                                                        feature_index=cell_no,
                                                        expression_threshold=expression_threshold,
                                                        redundant_genes=redundant
                                                        ).keys()))
    # print(cluster_signatures.shape)
    dataframes.append(cluster_signatures)

sigs = pd.concat(dataframes, axis=0)
print(sigs.shape)
print(total_cells)

        

1232
1074
791
727
695
615
571
541
491
406
398
368
314
293
220
201
199
148
86
(9370, 2)
9370


In [16]:
def process_clusters(cluster_table:dict[str,ad.AnnData], redundant_genes:set[str], expression_threshold=0.0, verbose=False) -> pd.DataFrame:
    """Given a dictionary of clusters and a set of redundant genes, calculates the gene_signature on a cell by cell basis.

    Args:
        cluster_table (dict[str,ad.AnnData]): Holds the cluster data with cluster names as keys.
        redundant_genes (set[str]): A set of separately calculated genes to exclude from signatures.
        expression_threshold (float, optional): Only count genes with expression levels greater than this number. Defaults to 0.0.
        verbose (bool, optional): Defaults to False.

    Returns:
        pd.DataFrame: A dataframe with cells as the index and columns for cluster name and signature (as a space separated string).
    """
    dataframes :list[pd.DataFrame] = []
    total_cells=0
    for cluster_name in cluster_table:
        cluster_signatures = pd.DataFrame(columns = ['cluster','signature'], index=cluster_table[cluster_name].obs.index)
        cluster_adata = cluster_table[cluster_name]
        n_cells, _ = cluster_adata.shape
        if verbose:
            print(f"cluster {cluster_name} has {n_cells} cells.")
        total_cells+=n_cells
        for cell_no in range(n_cells):
            cluster_signatures.iloc[cell_no,0] = cluster_name
            cluster_signatures.iloc[cell_no,1] = " ".join(list(get_gene_signature(adata=cluster_adata, 
                                                            feature_index=cell_no,
                                                            expression_threshold=expression_threshold,
                                                            redundant_genes=redundant
                                                            ).keys()))
        if verbose:
            print(cluster_signatures.head())
        dataframes.append(cluster_signatures)

    sigs = pd.concat(dataframes, axis=0)
    assert sigs.shape[0] == total_cells # sanity check to ensure that all cells are being processed
    if verbose:
        print(f"Processed {total_cells} to produce a dataframe with dimensions {sigs.shape}.")
    return sigs

redundant = find_common_genes(hv, expression_threshold=expression_threshold)
sigs_pd = process_clusters(cluster_table=cluster_table, redundant_genes=redundant,expression_threshold=1.5) 


In [17]:
b = sigs_pd[sigs_pd.cluster=='0']
b.count()

cluster      1232
signature    1232
dtype: int64

In [18]:
# write signatures to disk
sigs_pd.to_csv("../data/sigs.csv")

In [19]:
sigs_pd.to_parquet('../data/sigs.parquet')